# Patent Miner (Lean Runner)

This notebook is part of a complete patent discovery & analytics system.

## Setup
1. Create a `.env` file with: `PATENTSVIEW_API_KEY=your_key`
2. Install dependencies: `pip install -r requirements.txt`
3. Run the cells below to discover patents (1975-2005)

## Complete Workflow
1. **Discovery** (this notebook) → `patent_discoveries_*.json`
2. **Analytics** → Run `streamlit run streamlit_app.py`
3. **Export** → Dashboard includes CSV/JSON download

## Search Parameters
- **Years:** 1975-2005 (expanded from 1995-2005)
- **Keywords:** portable, sensor
- **Assignees:** individuals only
- **Status:** expired patents only


In [1]:
from dotenv import load_dotenv
import json
from datetime import datetime
from pathlib import Path

from patent_discovery import PatentDiscoveryError, discover_patents, save_discovery_diagnostics
from patent_miner_config import DEFAULT_CONFIG, build_config

# Load environment variables from .env file
load_dotenv()

CONFIG = build_config(DEFAULT_CONFIG)
OUTPUT_DIR = Path(CONFIG.get('output_dir', './patent_intelligence_vault/'))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print('Configuration loaded')
print('Provider:', CONFIG['patent_search']['provider'])
print('Keywords:', CONFIG['patent_search']['keywords'])
print('Num results:', CONFIG['patent_search']['num_results'])


Configuration loaded
Provider: patentsview_patentsearch
Keywords: ['portable', 'sensor']
Num results: 500


In [2]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

try:
    patents, diagnostics = discover_patents(CONFIG)

    discoveries_path = OUTPUT_DIR / f'patent_discoveries_{timestamp}.json'
    with open(discoveries_path, 'w', encoding='utf-8') as handle:
        json.dump(patents, handle, indent=2, ensure_ascii=False)

    diagnostics_path = save_discovery_diagnostics(str(OUTPUT_DIR), diagnostics, timestamp)

    print('Discovery complete')
    print('Patents found:', len(patents))
    print('Discoveries file:', discoveries_path)
    print('Diagnostics file:', diagnostics_path)

    if patents:
        print('\nTop 5 patents:')
        for idx, patent in enumerate(patents[:5], 1):
            print(f"{idx}. {patent.get('patent_number', 'N/A')} | {patent.get('title', 'Untitled')}")

except PatentDiscoveryError as exc:
    diagnostics = exc.diagnostics or {'status': 'failed', 'errors': [str(exc)]}
    diagnostics_path = save_discovery_diagnostics(str(OUTPUT_DIR), diagnostics, timestamp)
    print('Discovery failed')
    print('Code:', exc.code)
    print('Message:', exc.message)
    print('Diagnostics file:', diagnostics_path)
    raise


Discovery complete
Patents found: 19
Discoveries file: patent_intelligence_vault/patent_discoveries_20260216_183710.json
Diagnostics file: patent_intelligence_vault/discovery_diagnostics_20260216_183710.json

Top 5 patents:
1. 4516213 | Multiple rate metering system
2. 4817628 | System and method for evaluating neurological function controlling muscular movements
3. 4950997 | Diagnostic testing device for seed tube sensors
4. 5012411 | Apparatus for monitoring, storing and transmitting detected physiological information
5. 5059287 | Portable water distiller
